In [ ]:
# IMPORT LIBRARIES
import json
import matplotlib.pyplot as plt
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt
from sklearn import svm
import random
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
import collections

In [ ]:
# GLOBAL FUNCTIONS

def changeclass(input):
    temp = set(input)
    dict_classes = {k: i for i, k in enumerate(temp)}
    num_classes = [dict_classes[c] for c in input]
    return num_classes

def showclass(input):
    labels, values = zip(*collections.Counter(input).items())
    indexes = np.arange(len(labels))
    plt.bar(values, indexes, 1)
    plt.yticks(indexes, labels)
    plt.plot(values,labels,linewidth=3)
    plt.show()
    
def showaccuracy(per_class_accuracy, per_class_accuracy_ET):
    for i in range(0,21):
        Count = collections.Counter(per_class_accuracy[i])
        if Count[0] == 0 and Count[1] == 0:
            print("%d : No Samples" % i)
        else:
            print("%d : %0.2f , True: %d , False: %d" % (i,round((Count[1]/(Count[1] + Count[0])) * 100,2), Count[1], Count[0]))


    print("\n\n")        
    for i in range(0,21):
        Count_ET = collections.Counter(per_class_accuracy_ET[i])
        if Count_ET[0] == 0 and Count_ET[1] == 0:
            print("%d : No Samples" % i)
        else:
            print("%d : %0.2f , True: %d , False: %d" % (i,round((Count_ET[1]/(Count_ET[1] + Count_ET[0])) * 100,2), Count_ET[1], Count_ET[0]))

In [ ]:
# GLOBAL VARIABLES
N_BINS = 60                                   # USED IN NORMALIZING THE DATA TO COMMON FORM
TRAINED_DATA = 25000                          # HOW MANY DATA DO WE WANT TO TRAIN
COMBINED_CLASSIFICATION_SAMPLES = 30000       # HOW MANY SAMPLES IN COMBINED CLASSIFICATION

In [ ]:
# READ THE DATA
data = json.load(open('MoNA-export-GC-MS_Spectra.json', errors='ignore'))

In [ ]:
# CREAT CLEAN_DATA ONLY WITH SUBCLASS, CLASS, SUPERCLASS, SPECTRUM [VALUE, TEMPERATURE] INFORMATIONS
#we only care about those features and labels since or objective is to classify data(spectrum) by labels (subclass,class,superclass)
clean_data = []
#we add the data to the dictionary "clean_data"

for d in data:
  dict_i = {'class': '', 'subclass': '', 'superclass': ''}
  for c in d['compound'][0]['classification']:
    if c['name'] == 'class':
        dict_i['class'] = c['value']
    if c['name'] == 'subclass':
            dict_i['subclass'] = c['value']
    if c['name'] == 'superclass':
            dict_i['superclass'] = c['value']
    dict_i['spectrum'] = d['spectrum']
    clean_data.append(dict_i)

In [ ]:
# CREAT CLEAN_DATA_BINNED WHICH NORMALIZE DATA TO COMMON FORM (EVERY 'POINT' HAS 60 VALUES OF 'PEAKS' IN SPECTRUM)
plot = False #activate or deactivate ploting

bins = np.linspace(0, 300, N_BINS) #our data mostly goes from 50 to 250 but has some exceptions so we decided to do it from 0 to 300
clean_data_binned = []
#we create bins to divide the information in equaly sized groups
for d in clean_data:
    binned_data_i = {k: d[k] for k in ['class', 'superclass', 'subclass']}
    spec = np.array([[float(sp) for sp in s.split(':')] for s in d['spectrum'].split()])
    n_bin = np.digitize(spec[:, 0], bins)
    all_values_for_bin_x = [np.array(spec[:, 0][n_bin == i]) for i in range(1, len(bins))]
    all_values_for_bin_x = [v.mean() if len(v) else 0 for v in all_values_for_bin_x]
    all_values_for_bin = [np.array(spec[:, 1][n_bin == i]) for i in range(1, len(bins))]
    all_values_for_bin = [v if len(v) else np.array([0]) for v in all_values_for_bin ]
    all_values_for_bin_max = [v.max() for v in all_values_for_bin]
    binned_data_i['spectrum'] = [all_values_for_bin_x, all_values_for_bin_max]
    clean_data_binned.append(binned_data_i)
    if plot:
        plt.hist(all_values_for_bin_x, bins=bins, weights=all_values_for_bin_max)
        plt.plot(spec[:, 0], spec[:, 1], 'r')
        plt.show()

In [ ]:
# CREATING DICTIONARY FOR OUR FUNTIONS
spectrum = [c['spectrum'][1] for c in clean_data_binned]
superclass_d = [c['superclass'] for c in clean_data_binned]
classes_d = [c['class'] for c in clean_data_binned]
subclasses_d = [c['subclass'] for c in clean_data_binned]

# USING DEFINED HIGHER FUNCTION 'CHANGECLASS'
superclass = changeclass(superclass_d) 
classes = changeclass(classes_d)
subclasses = changeclass(subclasses_d)

In [ ]:
# GRAPHS WITH AMOUNT OF EACH TYPE OF SUBCLASS, CLASS AND SUPERCLASS IN THE DATA
#showclass(superclass_d)
#showclass(classes_d)
#showclass(subclasses_d)

In [ ]:
# RANDOMLY SHUFFLE DATA
packed_data = list(zip(spectrum, superclass, classes, subclasses))
random.shuffle(packed_data)
spectrum, superclass, classes, subclasses = zip(*packed_data)

In [ ]:
# SPLIT DATA TO TRAIN_DATA AND TEST_DATA
spectrum_train   = spectrum[:TRAINED_DATA]
spectrum_test    = spectrum[TRAINED_DATA:]
superclass_train = superclass[:TRAINED_DATA]
superclass_test  = superclass[TRAINED_DATA:]
classes_train    = classes[:TRAINED_DATA]
classes_test     = classes[TRAINED_DATA:]
subclasses_train = subclasses[:TRAINED_DATA]
subclasses_test  = subclasses[TRAINED_DATA:]

In [ ]:
# SUPERCLASSES TRAINING PART
#
#
# 20 Different classes.

In [ ]:
# DECISION TREE CLASSIFIER
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
model_tree = model_tree.fit(spectrum_train, superclass_train)
print("Trees model without setting parameters:")
model_tree_score = model_tree.score(spectrum_test, superclass_test)
print(model_tree_score)

In [ ]:
# NAIVE BAYES TRAINING SUPERCLASSES
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(spectrum_train, superclass_train)
print("Naive Bayes model without setting parameters:")
model_nb_score = model_nb.score(spectrum_test, superclass_test)
print(model_nb_score)

In [ ]:
# K-NN TRAINING SUPERCLASSES # LONGEST
from sklearn.neighbors import KNeighborsClassifier  
model_knn = KNeighborsClassifier()
model_knn.fit(spectrum_train, superclass_train)
print("KNN model without setting parameters:")
model_knn_score = model_knn.score(spectrum_test, superclass_test)
print(model_knn_score)

In [ ]:
# SVM KERNEL TRAINING SUPERCLASSES
model_svm = svm.SVC(gamma='scale',probability=True)
model_svm.fit(spectrum_train, superclass_train)
print("SVM model without setting parameters:")
model_svm_score = model_svm.score(spectrum_test, superclass_test)
print(model_svm_score)

In [ ]:
# EXTRA TREES CLASSIFIER
from sklearn.ensemble import ExtraTreesClassifier
model_rf = ExtraTreesClassifier(n_estimators = 100)
model_rf.fit(spectrum_train, superclass_train)
print("Extra Trees model without setting parameters:")
model_rf_score = model_rf.score(spectrum_test, superclass_test)
print(model_rf_score)

In [ ]:
Weight_Label = np.zeros((5,1))
Occuring_Label = np.zeros((21, 2))
#The sum of all scores(total score) is used as a way of normalizing the  score of each model given_model_score/total score

total_score = model_nb_score + model_knn_score + model_svm_score + model_rf_score + model_tree_score

Weight_Label[0,0] = model_nb_score / total_score
Weight_Label[1,0] = model_knn_score / total_score
Weight_Label[2,0] = model_svm_score / total_score
Weight_Label[3,0] = model_rf_score / total_score
Weight_Label[4,0] = model_tree_score / total_score

for i in range(0,21):
    Occuring_Label[i,0] = i

In [ ]:
FinalPrediction = []
Predict_Label = []

per_class_accuracy = {k: [] for k in list(set(superclass_test))}
per_class_accuracy_ET = {k: [] for k in list(set(superclass_test))}

for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    Predict_NB = model_nb.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_KNN = model_knn.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_SVM = model_svm.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_RF = model_rf.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_TREE = model_tree.predict(np.array(spectrum_test[i]).reshape(1, -1))
    
    Predict_Label = [Predict_NB,Predict_KNN,Predict_SVM,Predict_RF,Predict_TREE]
    #We use the score of each classifiers as weight for them
    #to give importance to each classifier and all those importance together 
    
    
    for j in range(0,21):
        for k in range(0,4):
            if Predict_Label[k] == j:
                Occuring_Label[j,1] += Occuring_Label[j,1] + Weight_Label[k,0]
    Score = np.where(Occuring_Label[:,1] == np.amax(Occuring_Label[:,1], axis=0))[0]

    FinalPrediction.append(Score[0])
    for j in range(0,21):
        Occuring_Label[j,1] = 0
        
    accuracy = Score[0] == superclass_test[i] #boolean
    accuracy_ET = Predict_RF == superclass_test[i] #boolean
    per_class_accuracy[superclass_test[i]].append(accuracy) #accuracy of each individual label
    per_class_accuracy_ET[superclass_test[i]].append(accuracy_ET) #accuracy of each individual label

per_class_accuracy_ET_Done = {k: [v[0] for v in val] for k, val in per_class_accuracy_ET.items()} #retranslate the variable

In [ ]:
# COMPUTE THE PERCENTAGE OF ACCURACY FOR EACH LABEL
showaccuracy(per_class_accuracy, per_class_accuracy_ET_Done)

In [ ]:
#compute each score by comparing ground truth with predictions
pluscounter = 0
minuscounter = 0
for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    if(FinalPrediction[i] == superclass_test[i]):
        pluscounter += 1
    else:
        minuscounter += 1

print(pluscounter/COMBINED_CLASSIFICATION_SAMPLES)
print('\n')

pluscounter = 0
minuscounter = 0
model = model_rf.predict(spectrum_test[:COMBINED_CLASSIFICATION_SAMPLES])

for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    #print(FinalPrediction[i], superclass_test[i], model[i])
    if(model[i] == superclass_test[i]):
        pluscounter += 1
    else:
        minuscounter += 1

print(pluscounter/COMBINED_CLASSIFICATION_SAMPLES)

In [ ]:
# CLASSES TRAINING PART
#
#
# 203 Different classes.

In [ ]:
# DECISION TREE CLASSIFIER
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
model_tree = model_tree.fit(spectrum_train, classes_train)
print("Trees model without setting parameters:")
model_tree_score = model_tree.score(spectrum_test, classes_test)
print(model_tree_score)

In [ ]:
# NAIVE BAYES TRAINING SUPERCLASSES
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(spectrum_train, classes_train)
print("Naive Bayes model without setting parameters:")
model_nb_score = model_nb.score(spectrum_test, classes_test)
print(model_nb_score)

In [ ]:
# K-NN TRAINING SUPERCLASSES
from sklearn.neighbors import KNeighborsClassifier  
model_knn = KNeighborsClassifier()
model_knn.fit(spectrum_train, classes_train)
print("KNN model without setting parameters:")
model_knn_score = model_knn.score(spectrum_test, classes_test)
print(model_knn_score)

In [ ]:
# SVM KERNEL TRAINING SUPERCLASSES
model_svm = svm.SVC(gamma='scale',probability=True)
model_svm.fit(spectrum_train, classes_train)
print("SVM model without setting parameters:")
model_svm_score = model_svm.score(spectrum_test, classes_test)
print(model_svm_score)

In [ ]:
# EXTRA TREES CLASSIFIER
from sklearn.ensemble import ExtraTreesClassifier
model_rf = ExtraTreesClassifier(n_estimators = 100)
model_rf.fit(spectrum_train, classes_train)
print("Extra Trees model without setting parameters:")
model_rf_score = model_rf.score(spectrum_test, classes_test)
print(model_rf_score)

In [ ]:
Weight_Label = np.zeros((5,1))
Occuring_Label = np.zeros((203, 2))

Weight_Label[0,0] = model_nb_score
Weight_Label[1,0] = model_knn_score
Weight_Label[2,0] = model_svm_score
Weight_Label[3,0] = model_rf_score
Weight_Label[4,0] = model_tree_score

for i in range(0,203):
    Occuring_Label[i,0] = i

In [ ]:
FinalPrediction = []
Predict_Label = []

per_class_accuracy = {k: [] for k in list(set(superclass_test))}
per_class_accuracy_ET = {k: [] for k in list(set(superclass_test))}

for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    Predict_NB = model_nb.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_KNN = model_knn.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_SVM = model_svm.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_RF = model_rf.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_TREE = model_tree.predict(np.array(spectrum_test[i]).reshape(1, -1))
    
    Predict_Label = [Predict_NB,Predict_KNN,Predict_SVM,Predict_RF,Predict_TREE]
    #We use the score of each classifiers as weight for them
    #to give importance to each classifier and all those importance together 
    
    
    for j in range(0,203):
        for k in range(0,4):
            if Predict_Label[k] == j:
                Occuring_Label[j,1] += Occuring_Label[j,1] + Weight_Label[k,0]
    Score = np.where(Occuring_Label[:,1] == np.amax(Occuring_Label[:,1], axis=0))[0]

    FinalPrediction.append(Score[0])
    for j in range(0,203):
        Occuring_Label[j,1] = 0
        
    accuracy = Score[0] == superclass_test[i] #boolean
    accuracy_ET = Predict_RF == superclass_test[i] #boolean
    per_class_accuracy[superclass_test[i]].append(accuracy) #accuracy of each individual label
    per_class_accuracy_ET[superclass_test[i]].append(accuracy_ET) #accuracy of each individual label

per_class_accuracy_ET_Done = {k: [v[0] for v in val] for k, val in per_class_accuracy_ET.items()} #retranslate the variable

In [ ]:
pluscounter = 0
minuscounter = 0
for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    if(FinalPrediction[i] == classes_test[i]):
        pluscounter += 1
    else:
        minuscounter += 1
                
print(pluscounter/COMBINED_CLASSIFICATION_SAMPLES)
print('\n')

pluscounter = 0
minuscounter = 0
model = model_rf.predict(spectrum_test[:COMBINED_CLASSIFICATION_SAMPLES])

for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    #print(FinalPrediction[i], superclass_test[i], model[i])
    if(model[i] == classes_test[i]):
        pluscounter += 1
    else:
        minuscounter += 1
        
print(pluscounter/COMBINED_CLASSIFICATION_SAMPLES)   

In [ ]:
# SUBCLASSES TRAINING PART
#
#
# 375 Different classes.

In [ ]:
# DECISION TREE CLASSIFIER
from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
model_tree = model_tree.fit(spectrum_train, subclasses_train)
print("Trees model without setting parameters:")
model_tree_score = model_tree.score(spectrum_test, subclasses_test)
print(model_tree_score)

In [ ]:
# NAIVE BAYES TRAINING SUPERCLASSES
from sklearn.naive_bayes import GaussianNB
model_nb = GaussianNB()
model_nb.fit(spectrum_train, subclasses_train)
print("Naive Bayes model without setting parameters:")
model_nb_score = model_nb.score(spectrum_test, subclasses_test)
print(model_nb_score)

In [ ]:
# K-NN TRAINING SUPERCLASSES
from sklearn.neighbors import KNeighborsClassifier  
model_knn = KNeighborsClassifier()
model_knn.fit(spectrum_train, subclasses_train)
print("KNN model without setting parameters:")
model_knn_score = model_knn.score(spectrum_test, subclasses_test)
print(model_knn_score)

In [ ]:
# SVM KERNEL TRAINING SUPERCLASSES
model_svm = svm.SVC(gamma='scale',probability=True)
model_svm.fit(spectrum_train, subclasses_train)
print("SVM model without setting parameters:")
model_svm_score = model_svm.score(spectrum_test, subclasses_test)
print(model_svm_score)

In [ ]:
# EXTRA TREES CLASSIFIER
from sklearn.ensemble import ExtraTreesClassifier
model_rf = ExtraTreesClassifier(n_estimators = 100)
model_rf.fit(spectrum_train, subclasses_train)
print("Extra Trees model without setting parameters:")
model_rf_score = model_rf.score(spectrum_test, subclasses_test)
print(model_rf_score)

In [ ]:
Weight_Label = np.zeros((5,1))
Occuring_Label = np.zeros((375, 2))

Weight_Label[0,0] = model_nb_score
Weight_Label[1,0] = model_knn_score
Weight_Label[2,0] = model_svm_score
Weight_Label[3,0] = model_rf_score
Weight_Label[4,0] = model_tree_score

for i in range(0,375):
    Occuring_Label[i,0] = i

In [ ]:
FinalPrediction = []
Predict_Label = []

per_class_accuracy = {k: [] for k in list(set(superclass_test))}
per_class_accuracy_ET = {k: [] for k in list(set(superclass_test))}

for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    Predict_NB = model_nb.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_KNN = model_knn.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_SVM = model_svm.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_RF = model_rf.predict(np.array(spectrum_test[i]).reshape(1, -1))
    Predict_TREE = model_tree.predict(np.array(spectrum_test[i]).reshape(1, -1))
    
    Predict_Label = [Predict_NB,Predict_KNN,Predict_SVM,Predict_RF,Predict_TREE]
    #We use the score of each classifiers as weight for them
    #to give importance to each classifier and all those importance together 
    
    
    for j in range(0,375):
        for k in range(0,4):
            if Predict_Label[k] == j:
                Occuring_Label[j,1] += Occuring_Label[j,1] + Weight_Label[k,0]
    Score = np.where(Occuring_Label[:,1] == np.amax(Occuring_Label[:,1], axis=0))[0]

    FinalPrediction.append(Score[0])
    for j in range(0,375):
        Occuring_Label[j,1] = 0
        
    accuracy = Score[0] == superclass_test[i] #boolean
    accuracy_ET = Predict_RF == superclass_test[i] #boolean
    per_class_accuracy[superclass_test[i]].append(accuracy) #accuracy of each individual label
    per_class_accuracy_ET[superclass_test[i]].append(accuracy_ET) #accuracy of each individual label

per_class_accuracy_ET_Done = {k: [v[0] for v in val] for k, val in per_class_accuracy_ET.items()} #retranslate the variable

In [ ]:
pluscounter = 0
minuscounter = 0
for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    if(FinalPrediction[i] == subclasses_test[i]):
        pluscounter += 1
    else:
        minuscounter += 1
                
print(pluscounter/COMBINED_CLASSIFICATION_SAMPLES)
print('\n')

pluscounter = 0
minuscounter = 0
model = model_rf.predict(spectrum_test[:COMBINED_CLASSIFICATION_SAMPLES])

for i in range(0,COMBINED_CLASSIFICATION_SAMPLES):
    #print(FinalPrediction[i], superclass_test[i], model[i])
    if(model[i] == subclasses_test[i]):
        pluscounter += 1
    else:
        minuscounter += 1
        
print(pluscounter/COMBINED_CLASSIFICATION_SAMPLES)   